In [60]:
from character_embeddings import CharacterEmbeddings
from model import GermanHateSpeechModel
import pandas as pd
import os
from tqdm import tqdm
import torch
import numpy as np
from fuzzywuzzy import fuzz

In [94]:
def recovery_rate_(list1, list2, list3):
    src_count = 1
    def_count = 0
    for src,target in zip(list1, list2):
        if src in list1 and src not in list2:
            src_count+=1
        if src in list1 and src not in list2 and src in list3:
            def_count+=1
    if src_count > 1:
        src_count -=1
    return def_count/ src_count 


In [97]:
basepath = "./datasets"
results = []
for filename in tqdm(os.listdir(f"{basepath}")):
    if "defense" in filename and "summary" not in filename:
        path = f"{basepath}/{filename}"
        dataset = pd.read_csv(path)
        
        recovery_rate = []
        
        for idx,row in dataset.iterrows():
            original_text, perturbed_text, defended_text = (row['original_text'], 
                                                            row['perturbed_text'], row['defended_text'])
            recovery_rate.append(recovery_rate_(original_text.split(" "), 
                                                        perturbed_text.split(" "), defended_text.split(" ")))
        results.append({
                        "token_recovery_rate_mean":np.mean(recovery_rate), 
                        "token_recovery_rate_median":np.median(recovery_rate),
                        "adverserial_source": filename 
                       })

100%|██████████| 9/9 [00:00<00:00, 11.89it/s]


In [98]:
pd.DataFrame(results)

,token_recovery_rate_mean,token_recovery_rate_median,adverserial_source
0,0.545741,0.5,results-germeval_whitebox_charlevel_attack_def...
1,0.008235,0.0,results-hasoc_whitebox_charlevel_attack_defens...
2,0.673529,1.0,results-germeval_blackbox_charlevel_attack_def...
3,0.008204,0.0,results-hasoc_blackbox_charlevel_attack_defens...
